# Llamafile x Autogen

This notebook aims to show how you can run mixtral 8x7B with Autogen.

**Step to run locally:**
- Download a llamafile of the [HuggingFace Repo](https://huggingface.co/award40/mixtral-8x7b-instruct-v0.1.Q3_K_M.llamafile/tree/main) - no installations needed, read the model card.  
- `chmod +x mixtral-8x7b-instruct-v0.1.Q3_K_M.llamafile`
- `./mixtral-8x7b-instruct-v0.1.Q3_K_M.llamafile --port 8081`
<!-- ./llamafile -m mixtral-8x7b-instruct-v0.1.Q3_K_M.gguf -->

Like many of you, i am GPU poor. The goal behind this approach was to have easy access to a good opensourced model with limited GPU resources, like a Macbook Pro M1 32GB.
It's not the full model, but it's the most feasible given the resource constraints - see [here](https://huggingface.co/TheBloke/Mixtral-8x7B-Instruct-v0.1-GGUF#provided-files) for notes on performance

In [1]:
import os
import shutil
import autogen 
from autogen import AssistantAgent, UserProxyAgent


work_dir = "./coding"
cache_dir = "./.cache"

if os.path.exists(cache_dir):
    print('Deleting previous work...')
    shutil.rmtree(cache_dir)
    shutil.rmtree(work_dir)

In [2]:
# API Configurations
config_list=[
    {
        "base_url": "http://127.0.0.1:8081/v1",
        "api_key": "NULL", # just a placeholder
    }
]

llm_config_mistral={
    "config_list": config_list,
}

llm_config_mistral

{'config_list': [{'base_url': 'http://127.0.0.1:8081/v1', 'api_key': 'NULL'}]}

In [ ]:
# Instructions for the two agents
system_messages = {
    "USER_PROXY_SYSTEM_MESSAGE": (
        """
        Your job is to act as an natural language AI interface between the human use and the coding assistant."
        Your job is NOT to take part in conversations, but to act as an intermediary for the human and the coding assistant.
        Say TERMINATE when all tasks are finished.
        """
    ),
    "CODING_ASSISTANT_SYSTEM_MESSAGE" : (
        """
        As a expert programmer, your role involves generating answer a question. 
        \n\n
        When a task is presented, you should:
        Step 1: Determine the user's intent by considering the question and its context. The intent might involve generating code or answering a question, or both.
        Step 2: Formulate a response based on the identified intent.
        Step 3: If your code doesn't work, use the subsequent messages to refactor and fix the issues.
        This approach will provide a well-informed basis for your responses.
         \n\n

        REMEMBER:
        - You MUST provide full working code for the user to run.
        - You MUST write code for in your answer while adhering to this format:
        ```language
        <code>
        ```
        - You MUST present the code in a single code block, do NOT split up into sections.
        - Do NOT engage in trivial conversations, just do your job!
        \n\n
        Say TERMINATE when all tasks are finished.
        """
    )
}

coding_assistant = AssistantAgent(
    name="coding_assistant",
    llm_config=llm_config_mistral,
    system_message=system_messages['CODING_ASSISTANT_SYSTEM_MESSAGE']
)

coding_runner = UserProxyAgent(
    name="coding_runner",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=5,
    is_termination_msg = lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    llm_config=llm_config_mistral,
    code_execution_config={
        "work_dir": work_dir, 
        "use_docker": False
    },
    system_message=system_messages['USER_PROXY_SYSTEM_MESSAGE']
)

### Initiate a chat with a simple python problem

In [ ]:
# prevent an infitinate gratitude loop
def get_additional_termination_notice():
    """Extra instructions for terminating when goal is finished."""
    
    termination_notice = (
        '\n\nDo not show appreciation in your responses, say only what is necessary. '
        'if "Thank you" or "You\'re welcome" are said in the conversation, then say TERMINATE '
        'to indicate the conversation is finished'
        # "Say TERMINATE when no further instructions are given to indicate the task is complete"
    )
    return termination_notice

In [ ]:
user_message = "Write a fibonacci sequence function in python and run it to print 20 numbers."
# user_message = user_message + f"{get_additional_termination_notice()}"

In [ ]:
coding_runner.initiate_chat(coding_assistant, 
                            message=user_message)